In [ ]:
%pip install pandas
%pip install stanfordnlp
%pip install senticnet
%pip install sentistrength
%pip install nltk
%pip install spacy
%pip install sklearn
%pip install numpy

# run this in the terminal
# python -m spacy download en_core_web_sm

In [17]:
import json
import pandas as pd
from stanfordcorenlp import StanfordCoreNLP
import requests
from senticnet.senticnet import SenticNet
from sentistrength import PySentiStr
import nltk
import spacy
from nltk.stem import WordNetLemmatizer
import re
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np


In [2]:
nltk.download('punkt')
# stanfordNLP = StanfordCoreNLP("http://localhost", port=8000, timeout=30000)
spacyNLP = spacy.load("en_core_web_sm")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# download the zip file from the link https://drive.google.com/file/d/1yvCpB2URy0iFjQPn3RmidNOryTlo6vHG/view?usp=share_link
# extract the zip file and place the folder in the same directory as this file then cd into the folder
# run the following command in the terminal to start the server
# java -mx4g -cp "*" edu.stanford.stanfordNLP.pipeline.StanfordCoreNLPServer -port {8000 or any port} -timeout 30000
# can speed it up by replace 4g with 8g (it represents the ram being used in gigs)
def lemmatize(text):
    # perform lemmatization
    lemmas = []
    output = stanfordNLP.annotate(text, properties={'annotators': 'tokenize,lemma', 'outputFormat': 'json'})
    output_dict = json.loads(output)
    tokens = output_dict['sentences'][0]['tokens']
    for token in tokens:
        lemmas.append(token['lemma'])
   
    return lemmas  

#### Converting the given JSON file into actual JSON format for easier readbility

In [ ]:
writeFile = open("Sarcasm_Headlines.json", "w")
writeFile.write("{ \"headlines\": [")
with open("Sarcasm_Headlines_Dataset.json") as readFile:
  for item in readFile:
    writeFile.write(item + ",")
# removed the final comma manually
writeFile.write("]}")
readFile.close()
writeFile.close()

# Preprocessing Stage

#### Reading the dataset and removing all article links as our goal is to analyze the headlines for sarcasm

In [ ]:
dataset = json.load(open("Sarcasm_Headlines.json"))
df = pd.DataFrame(dataset["headlines"])
df.drop(["article_link"], axis = 1, inplace = True)
df.head()

#### lemmatizing the dataset

In [ ]:
def lemmatizeDataset():
    for index, row in df.iterrows():
        sentence = row['headline']
        row['headline'] = lemmatize(sentence)

lemmatizeDataset()
df.head()

#### writing to a csv file to avoid having to perform pre-processing again

In [ ]:
df.to_csv('lemmatized.csv', index=False)

In [4]:
df = pd.read_csv("lemmatized.csv")
df.head()

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0


# Module 1 => Concept Level and Common Sense Knowledge
### ConceptNet
ConceptNet is a semantic network consisting of common-sense knowledge and concepts, represented<br> in the form of nodes (words or
short phrases) and labeled edges (relationships) between them.

In [4]:

# set the API endpoint and parameters
endpoint = 'http://api.conceptnet.io/c/en/'
params = {
    'filter': 'core',
    'limit': 1000
}
def conceptNet(sentence):
    # send a GET request to the API endpoint
    response = requests.get(endpoint + sentence, params=params)

    # parse the JSON response
    data = json.loads(response.text)
    edges = data['edges']
    edges.sort(key=lambda x: x['weight'], reverse=True)

    return edges

# Module 2 => Sentiment Score
### SentiStrength
SentiStrength is a sentiment lexicon that uses linguistic information and rules to detect<br>
sentiment strength in English text. SentiStrength provides positive and negative sentiment<br>
scores for each word. Both scores are integers from 1 to 5, where 1 signifies weak sentiment<br>
and 5 signifies strong sentiment.
<br>
polarity = positiveSentiment - negativeSentiment

### SenticNet
SenticNet is a resource for opinion mining that aims to create a collection of commonly<br> 
used common-sense concepts  with positive and negative sentiment scores. The sentiment <br>
score for each word is scaled from -1 to 1, where -1 signifies strongly negative sentiment,<br>
0 signifies neutral sentiment and 1 signifies strong positive sentiment.
<br> sentiment = score * 5 (in-order to keep it with sentiStrength)

### Rules of w_score (sentiment score) selection:
- if word belongs to SentiStrength || SenticNet => pick the score whichever exists
- if word belongs to SentiStrength && SenticNet => avg score of the lexicons
- else get the concepts from concept net to expand the meaning => select top 5 ranked and calculate the avg sentiment score

### Final Calculation
sum_pos_score = sum of all positive sentiment scores<br>
sum_neg_score = sum of all negative sentiment scores<br>
if sum_pos_score && sum_neg_score > 0, there is a contradiction in the sentence

In [5]:
sn = SenticNet()
def senticNetScore(word):
    try:
        polarityValue = sn.polarity_value(word)
        return float(polarityValue) * 5
    except KeyError:
        return None

In [6]:
senti = PySentiStr()
# got the jar file and data folder from the author (also reverse engineered the pysenti package to extract the jar file)
senti.setSentiStrengthPath('D:/Sarcasm_Detection-Feature_Selection/SentiStrengthCom.jar')
senti.setSentiStrengthLanguageFolderPath('D:/Sarcasm_Detection-Feature_Selection/SentStrength_Data')
def sentiStrengthScore(word):
    result = senti.getSentiment(word)
    return result

In [7]:
def wScore(word):
    senticNet = senticNetScore(word)
    sentiStrength = sentiStrengthScore(word)[0]
    if senticNet == None and sentiStrength == None:
        expansion = conceptNet(word)
        if len(expansion) == 0:
            return 0
        else:
            score = 0
            expansion = expansion[:5]
            for edge in expansion:
                score += wScore(edge['end']['label'])
            return score / 5
    elif senticNet == None:
        return sentiStrength
    elif sentiStrength == None:
        return senticNet
    else:
        return (senticNet + sentiStrength) / 2

In [8]:
def positiveScore(results):
    score = 0
    for result in results:
        if result > 0:
            score += result
    return score
def negativeScore(results):
    score = 0
    for result in results:
        if result < 0:
            score += result
    return score

# Module 3 => Sentence Coherence
Checking the coreference between subjects or objects of a sentence
<br> for two subjects w1 and w2, sentence is coherent if
- if w1 is antecedent of w2
- if w1 and w2 are identical pronouns
- if w1 and w2 are identical subjects
- w2 starts with the word "the" (Definite Noun Phrase)
- w2 starts with "this", "that", "these", "those" (Demonstrative Noun Phrases)
- if w1 and w2 are proper nouns

In [9]:
def extractSubject(sentence):
    doc = spacyNLP(sentence)
    subject = None
    for token in doc:
        if token.dep_ == "nsubj":
            subject = token.text
    return subject

In [10]:
def hasAntecedents(text):
    doc = spacyNLP(text)
    antecedents = []
    for token in doc:
        if token.dep_ == "nsubj" and token.head.pos_ == "VERB":
            for mention in doc.ents:
                if mention.start <= token.i < mention.end:
                    antecedents.append(mention.text)
    return True if len(antecedents) > 0 else False


In [11]:
pronounLemmatizer = WordNetLemmatizer()
def identicalPronouns(w1, w2):
    lemma1 = pronounLemmatizer.lemmatize(w1, 'n')
    lemma2 = pronounLemmatizer.lemmatize(w2, 'n')
    if lemma1 == lemma2:
        return True
    else:
        return False
    

In [12]:
def identicalSubjects(w1,w2):
    cleanedSubject1 = re.sub(r'[^a-zA-Z]', '', w1)
    cleanedSubject2 = re.sub(r'[^a-zA-Z]', '', w2)
    if cleanedSubject1 == cleanedSubject2:
        return True
    else:
        return False

In [13]:
def definiteNounPhraseFeature(text,w2):
    doc = nltk.word_tokenize(text)
    for i in range(len(doc)):
        if i-1 >= 0 and doc[i] == w2:
            if doc[i-1] == 'the':
                return True
    return False

In [14]:
def demonstrativeNounPhraseFeature(text,w2):
    doc = nltk.word_tokenize(text)
    for i in range(len(doc)):
        if doc[i] == w2:
            if i-1 >= 0 and doc[i-1] == 'this' or doc[i-1] == 'that' or doc[i-1] == 'these' or doc[i-1] == 'those':
                return True
    return False

In [15]:
def properNameFeature(w1,w2):
    taggedWords = nltk.pos_tag([w1,w2])
    proper = False
    for word, tag in taggedWords:
        if tag in ['NNP', 'NNPS']:
            proper = True
        else:
            proper = False
            break
    return proper

# Module 4 => Creation of Feature Vector

### N-Grams Features


In [14]:
headlines = df.drop('is_sarcastic',axis='columns')
sentences = df['headline']
def remove_symbols(line):
    return ''.join(ch for ch in line if ch.isalnum() or ch == " ")

def createNgrams(sentence):
    ngrams = []
    sentence = remove_symbols(sentence)
    tokens = nltk.word_tokenize(sentence)
    bigrams = list(nltk.bigrams(tokens))
    trigrams = list(nltk.trigrams(tokens))
    return tokens, bigrams, trigrams

### Creating Feature Space

In [16]:
def createFeatureSpace():
    ngrams = []
    featureSpace = pd.DataFrame()
    counter = 1
    for sentence in sentences:
        unigram, bigrams, trigrams = createNgrams(sentence)
        for word in unigram:
            ngrams.append(word)
        for word in bigrams:
            ngrams.append("{} {}".format(word[0], word[1]))
        for word in trigrams:
            ngrams.append("{} {} {}".format(word[0], word[1], word[2]))
        counter += 1
        print(counter, len(ngrams))
    ngrams = list(set(ngrams))
    data = np.zeros((len(sentences), len(ngrams)))
    print(len(data))
    # featureSpace = pd.DataFrame(data)
    return featureSpace
featureSpace = createFeatureSpace()
print(featureSpace.shape)
featureSpace.head()    

2 33
3 72
4 111
5 147
6 177
7 186
8 204
9 243
10 261
11 285
12 312
13 333
14 366
15 393
16 414
17 435
18 480
19 498
20 522
21 528
22 546
23 591
24 621
25 645
26 681
27 699
28 720
29 729
30 747
31 765
32 786
33 813
34 846
35 870
36 891
37 912
38 924
39 966
40 996
41 1023
42 1050
43 1092
44 1113
45 1134
46 1155
47 1170
48 1212
49 1242
50 1260
51 1269
52 1278
53 1308
54 1332
55 1368
56 1404
57 1434
58 1464
59 1488
60 1515
61 1548
62 1557
63 1569
64 1584
65 1620
66 1653
67 1677
68 1710
69 1758
70 1776
71 1794
72 1815
73 1839
74 1863
75 1884
76 1920
77 1950
78 1974
79 2001
80 2013
81 2034
82 2061
83 2082
84 2109
85 2145
86 2172
87 2202
88 2241
89 2268
90 2316
91 2352
92 2361
93 2415
94 2448
95 2469
96 2499
97 2523
98 2550
99 2577
100 2607
101 2631
102 2649
103 2682
104 2706
105 2733
106 2748
107 2772
108 2790
109 2817
110 2832
111 2862
112 2895
113 2928
114 2946
115 3009
116 3033
117 3063
118 3090
119 3144
120 3177
121 3213
122 3234
123 3240
124 3267
125 3294
126 3318
127 3342
128 3372
129 

KeyboardInterrupt: 

- Contradiction Feature: <br>
<emsp>We use two binary features Contra and Contra_Coher<br>
<emsp>Contra if headline has one sentence and contradiction in sentiment score occur
<br>
<emsp>Contra_Coher if headline has more than one sentence, contradiction of polarity and the headline is judged coherent<br>
- Sentiment Feature <br>
<emsp>Calculates the +ve and -ve score of the headline and then classify it as low/med/high
- Punctuation <br>
<emsp>We use 7 indicators<br><br>
    <emsp><emsp>1. Number of emoticons <br>
    <emsp><emsp>2. Number of repetitive sequence of punctuations<br>
    <emsp><emsp>3. Number of repetitive sequence of characters<br>
    <emsp><emsp>4. Number of capitalized word<br>
    <emsp><emsp>5. Number of slang and booster words<br>
    <emsp><emsp>6. Number of exclamation marks<br>
    <emsp><emsp>7. Number of idioms<br

In [ ]:
def calculate_scores(sentence):
    score=[]
    results = []
    for i in range(len(sentence)):
        results[i] = wScore(sentence[i])
    score[0]=positiveScore(results)
    score[1]=negativeScore(results)
    return score

def isContradiction(scores):
    if scores[0]!=0 and scores[1]!=0:
        return True
    return False

def checkCoherence(sentence):
    if hasAntecedents(".".join(sentence)) is not None:
        return True
    for i in range(len(sentence))-1:
        s1 = sentence[i]
        s2 = sentence[i+1]
        w1 = extractSubject(s1)
        w2 = extractSubject(s2)
        
        if identicalPronouns(w1,w2) or identicalSubjects(w1,w2) or definiteNounPhraseFeature(s2,w2) or demonstrativeNounPhraseFeature(s2,w2) or properNameFeature(w1,w2):
            return True
    
    return False
        

for sentence in sentences:
    # if only one sentence
    scores = []
    if '.' not in sentence:
        sentence = sentence.split(' ')
        for i in range(len(sentence)):
            sentence[i] = remove_symbols(sentence[i])
        scores = calculate_scores(sentence)
        if isContradiction(scores):
            df['contra'] = 1
        else:
            df['contra'] = 0
        df['contra_coher'] = 0
    # if more than one sentence
    else:
        sentence = sentence.split('.')
        if checkCoherence(sentence):
            for i in range(len(sentence)):
                temp_sentence = sentence[i]
                temp_sentence = temp_sentence.split(' ')
                for j in range(len(temp_sentence)):
                    temp_sentence[i] = remove_symbols(temp_sentence[i])
                scores = calculate_scores(temp_sentence)
                if isContradiction(scores):
                    df['contra_coher'] = 1
                else:
                    df['contra_coher'] = 0
        df['contra'] = 0
    ## implementing 4.4.3

    ## positive score
    if score[0] < -1:
        df['pos_low'] = 1
        df['pos_medium'] = 0
        df['pos_high'] = 0
    elif score[0] >= 0 and score[0]<=1:
        df['pos_low'] = 0
        df['pos_medium'] = 1
        df['pos_high'] = 0
    elif score[0] >= 2:
        df['pos_low'] = 0
        df['pos_medium'] = 0
        df['pos_high'] = 1
    else:
        df['pos_low'] = 0
        df['pos_medium'] = 0
        df['pos_high'] = 0
    
    ## negative score
    if score[1] < -1:
        df['neg_low'] = 1
        df['neg_medium'] = 0
        df['neg_high'] = 0
    elif score[1] >= 0 and score[1]<=1:
        df['neg_low'] = 0
        df['neg_medium'] = 1
        df['neg_high'] = 0
    elif score[1] >= 2:
        df['neg_low'] = 0
        df['neg_medium'] = 0
        df['neg_high'] = 1
    else:
        df['neg_low'] = 0
        df['neg_medium'] = 0
        df['neg_high'] = 0
df
